# **Exploring Static Data**
In this notebook, we will explore the static data provided by ATAC. Below is the entity relationship diagram for the static data. Here is the [link](https://drawsql.app/teams/atacontitan/diagrams/static-data) to the drawsql diagram.

![Static Data ERD](figures/er_diagram.png)

In [1]:
# Libraries
import pandas as pd
from google.transit import gtfs_realtime_pb2
feed = gtfs_realtime_pb2.FeedMessage()

# Static data source
src = 'data/rome_static_gtfs_test'

In [2]:
agency = pd.read_csv(f'{src}/agency.txt')
calendar = pd.read_csv(f'{src}/calendar.txt')
calendar_dates = pd.read_csv(f'{src}/calendar_dates.txt')
routes = pd.read_csv(f'{src}/routes.txt')
shapes = pd.read_csv(f'{src}/shapes.txt')
stop_times = pd.read_csv(f'{src}/stop_times.txt', nrows=1000)
stops = pd.read_csv(f'{src}/stops.txt')
trips = pd.read_csv(f'{src}/trips.txt')

In [3]:
print(f'Unique ID for agency is "agency_id": {agency["agency_id"].duplicated().sum()}')
print(f'Unique ID for calendar is "service_id": {calendar["service_id"].duplicated().sum()}')
print(f'Unique ID for routes is "route_id": {routes["route_id"].duplicated().sum()}')
print(f'Unique ID for shapes is "shape_id" and "shape_pt_sequence": {shapes[["shape_id", "shape_pt_sequence"]].duplicated().sum()}')
print(f'Unique ID for stops is "stop_id": {stops["stop_id"].duplicated().sum()}')
print(f'Unique ID for trips is "trip_id": {trips["trip_id"].duplicated().sum()}')

Unique ID for agency is "agency_id": 0
Unique ID for calendar is "service_id": 0
Unique ID for routes is "route_id": 0
Unique ID for shapes is "shape_id" and "shape_pt_sequence": 0
Unique ID for stops is "stop_id": 0
Unique ID for trips is "trip_id": 0


,route_id,agency_id,route_short_name,route_long_name,route_type,route_url,route_color,route_text_color
0,211,OP1,211,NaN,3,NaN,NaN,NaN
1,C2,OP1,C2,NaN,3,NaN,NaN,NaN
2,62,OP1,62,NaN,3,NaN,NaN,NaN
3,105,OP1,105,NaN,3,NaN,NaN,NaN
4,766,OP1,766,NaN,3,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
427,4358,OP2,764,NaN,3,http://muovi.roma.it/percorso/js?query=764&cl=1,NaN,NaN
428,312,OP2,665,NaN,3,http://muovi.roma.it/percorso/js?query=665&cl=1,NaN,NaN
429,345,OP2,437,NaN,3,http://muovi.roma.it/percorso/js?query=437&cl=1,NaN,NaN
430,3632,OP2,235,NaN,3,http://muovi.roma.it/percorso/js?query=235&cl=1,NaN,NaN


In [34]:
unique_routes = pd.Series(trips.loc[trips['trip_id'].str.len() > 20, 'route_id'].unique())
routes[routes['route_id'].isin(unique_routes)]

,route_id,agency_id,route_short_name,route_long_name,route_type,route_url,route_color,route_text_color
325,10,OP2,025,NaN,3,http://muovi.roma.it/percorso/js?query=025&cl=1,NaN,NaN
326,11,OP2,030,NaN,3,http://muovi.roma.it/percorso/js?query=030&cl=1,NaN,NaN
327,4192,OP2,035,NaN,3,http://muovi.roma.it/percorso/js?query=035&cl=1,NaN,NaN
328,24,OP2,023,NaN,3,http://muovi.roma.it/percorso/js?query=023&cl=1,NaN,NaN
329,143,OP2,710,NaN,3,http://muovi.roma.it/percorso/js?query=710&cl=1,NaN,NaN
...,...,...,...,...,...,...,...,...
427,4358,OP2,764,NaN,3,http://muovi.roma.it/percorso/js?query=764&cl=1,NaN,NaN
428,312,OP2,665,NaN,3,http://muovi.roma.it/percorso/js?query=665&cl=1,NaN,NaN
429,345,OP2,437,NaN,3,http://muovi.roma.it/percorso/js?query=437&cl=1,NaN,NaN
430,3632,OP2,235,NaN,3,http://muovi.roma.it/percorso/js?query=235&cl=1,NaN,NaN


In [35]:
agency

,agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone,agency_fare_url
0,OP1,Atac,http://www.atac.roma.it,Europe/Rome,it,06 57003,NaN
1,OP2,Roma TPL,http://romatpl.it,Europe/Rome,it,06 57003,NaN
2,OP3,Trenitalia,http://www.trenitalia.com/,Europe/Rome,it,NaN,NaN


In [4]:
import networkx as nx
# Create a graph
G = nx.Graph()

# Add nodes
for _, stop in stops.iterrows():
    G.add_node(stop['stop_id'], name=stop['stop_name'], lat=stop['stop_lat'], lon=stop['stop_lon'])

print(G)

Graph with 8688 nodes and 0 edges


In [5]:
def get_trip_edges(row):
    """Given a row of stop_times, returns stop_id pairs for each edge in the trip"""
    return list(zip(row['stop_id'][:-1].values, 
                    row['stop_id'][1: ].values))

def flatten_list(l):
    return [item for sublist in l for item in sublist]

def get_edge_set(stop_times) -> set:
    edges = stop_times.groupby('trip_id').apply(get_trip_edges).to_list()
    edges = flatten_list(edges)
    return set(edges)

edges = get_edge_set(stop_times)
print(len(edges))


184


In [6]:
chunksize = 10_000
edge_set = set()
from tqdm import tqdm
for chunk in tqdm(pd.read_csv(f'{src}/stop_times.txt', chunksize=chunksize, usecols=['trip_id', 'stop_id'], dtype=str), total=531):
    edges = get_edge_set(chunk)
    edge_set.update(edges)

print(f'Found {len(edge_set)} edges in first pass')

# I am repeating the above code with a different chunksize cause I may have missed some edges where
# the a dataset ends and the next one begins
chunksize -= 2
for chunk in tqdm(pd.read_csv(f'{src}/stop_times.txt', chunksize=chunksize, usecols=['trip_id', 'stop_id'], dtype=str), total=531):
    edges = get_edge_set(chunk)
    edge_set.update(edges)

print(f'Found {len(edge_set)} edges in second pass')

  0%|          | 2/531 [00:00<01:08,  7.74it/s]

100%|██████████| 531/531 [00:45<00:00, 11.61it/s]


Found 11166 edges in first pass


100%|██████████| 531/531 [00:45<00:00, 11.59it/s]

Found 11166 edges in second pass


In [7]:
G.add_edges_from(edge_set)
print(G)

Graph with 8688 nodes and 11154 edges


In [19]:
edge_df = pd.DataFrame(G.edges, columns=['node1', 'node2'], dtype=str)
# add id
# edge_df['id'] = edge_df.index
# edge_df = edge_df[['id', 'source', 'target']]
edge_df.to_parquet('data/edge_df.parquet')
edge_df.head()

,node1,node2
0,00213,ROME6558
1,00213,75141
2,00213,70650
3,00213,78475
4,00213,70651


In [9]:
# nodes without neighbors
pd.DataFrame(G.degree())[1].value_counts()

2     5313
3     1609
4      756
0      360
5      301
6      165
7       67
8       40
1       27
9       25
10       9
11       6
15       3
14       2
13       2
12       2
16       1
Name: 1, dtype: int64